In [13]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline

In [14]:
anes = pd.read_csv('data/anes_pilot_2020ets_csv.csv')

In [15]:
anes.head()

,V1,StartDate,EndDate,_v1,RecordedDate,ResponseId,qmetadata_Browser,qmetadata_Version,_v2,qmetadata_Resolution,...,rr_scale,white,black,namer,asian,hpi,mixed,race7,vidx,vidknow
0,1451,4/12/2020 17:04,4/12/2020 17:23,1131,4/12/2020 17:23,R_3oBJppI62KNh4MC,Chrome,81.0.4044.96,Android 9,360x760,...,9,1,0,0,0,0,0,1,2.0,1
1,2875,4/16/2020 10:14,4/16/2020 11:06,3170,4/16/2020 11:06,R_1jdLlxoKBpgExZS,Firefox,52,Windows NT 5.1,1024x768,...,16,1,0,0,0,0,0,1,3.0,0
2,859,4/17/2020 21:34,4/17/2020 22:32,3490,4/17/2020 22:32,R_ClTXw6WOcfMOLWp,Chrome,81.0.4044.111,Android 9,360x760,...,6,0,0,0,0,0,0,5,3.0,0
3,2031,4/16/2020 17:29,4/16/2020 18:34,3885,4/16/2020 18:34,R_3sjr4zeG5BbBJa0,Chrome,78.0.3904.96,Android 7.1.1,320x570,...,13,1,0,0,0,0,0,1,3.0,1
4,587,4/16/2020 15:20,4/16/2020 15:59,2330,4/16/2020 15:59,R_21H8jh0vE4GUqI5,Chrome,80.0.3987.132,Windows NT 10.0,1920x1080,...,4,0,1,0,0,0,0,2,2.0,0


In [16]:
anes.shape

(3080, 470)

# Dropping Columns 

In [17]:
#make a copy of the dataframe 
data = anes.copy(deep = True)

In [18]:
#look at all the column names
for col in data.columns:
    print(col)

V1
StartDate
EndDate
_v1
RecordedDate
ResponseId
qmetadata_Browser
qmetadata_Version
_v2
qmetadata_Resolution
follow
reg1
votemail1a
votemail1b
votecount
votemail2
voterid1
voterid2
turnout16a
turnout16a1
turnout16b
vote16
hopeful
afraid
outraged
angry
happy
worried
proud
irritated
nervous
meeting
moneyorg
protest
online
persuade
button
moneycand
argument
particip_none
talk1
talk2
talk3
fttrump1
ftobama1
ftbiden1
ftwarren1
ftsanders1
ftbuttigieg1
ftharris1
ftklobuchar1
ftpence1
ftyang1
ftpelosi1
ftrubio1
ftocasioc1
fthaley1
ftthomas1
ftfauci1
ftblack
ftwhite
fthisp
ftasian
ftillegal
ftfeminists
ftmetoo
fttransppl
ftsocialists
ftcapitalists
ftbigbusiness
ftlaborunions
ftrepublicanparty
ftdemocraticparty
primaryvote
vote20jb
vote20bs
cvote2020
apppres7
frnpres7
immpres7
econpres7
covidpres7
healthcarepres7
check
dtleader1
dtleader2
dtcares
dtdignif
dthonest
dtauth
dtdiv
dtknow
jbleader1
jbleader2
jbcares
jbdignif
jbhonest
jbauth
jbdiv
jbknow
bsleader1
bsleader2
bscares
bsdignif
bshonest


In [19]:
data.drop(inplace=True, columns=[ 'V1', 'StartDate', 'EndDate', '_v1', 'RecordedDate', 'ResponseId','qmetadata_Browser', 'qmetadata_Version', '_v2', 'qmetadata_Resolution', 'check'])






What columns I am dropping and why:
1. qmetadata_Broweser, qmetadata_Verrsion, _v2, qmetadata_Resolution, 
    these columns contains information about what browers repondents utlizied when taking the survey. For the purposes of my analysis, this information is not relevant. 
  
2. StartDate, EndDate and RecordedDate 
    these columns contain information about when the survey was conducted. Not needed 
 
3. 
    V1, _v1
        no information about these 

4. ResponseID
    unique ID for each respondent 
 
 5. 'check' column was a question that asked repondents to select a certaina nswer so the surveyors could ensure the respondent's device was working




In [20]:
data.isnull().values.any()

True

In [21]:
check_for_nan = data.isnull()
print (check_for_nan)

      follow   reg1  votemail1a  votemail1b  votecount  votemail2  voterid1  \
0      False  False       False       False      False      False     False   
1      False  False       False       False      False      False     False   
2      False  False       False       False      False      False     False   
3      False  False       False       False      False      False     False   
4      False  False       False       False      False      False     False   
...      ...    ...         ...         ...        ...        ...       ...   
3075   False  False       False       False      False      False     False   
3076   False  False       False       False      False      False     False   
3077   False  False       False       False      False      False     False   
3078   False  False       False       False      False      False     False   
3079   False  False       False       False      False      False     False   

      voterid2  turnout16a  turnout16a1  ...  rr_sc

# About Null Values

There are quite a few null values in this dataset, but they are important. In this dataset a null value likee 0 serves as an actual data point. For example, the column 'asian' contains information about the respondent's ethnoracial self-identification. If the respondent identifies as Asain, the value is 1 and if not, the value is 0. For these reasons, it is in my best interest to keep null values.

# Exploring Distributions

In [27]:
data['mis_covid1'].value_counts(normalize=True)

2    0.556169
1    0.442532
9    0.001299
Name: mis_covid1, dtype: float64

In [28]:
data['conf_covid1'].value_counts(normalize=True)

3    0.323701
4    0.239610
5    0.221429
2    0.152922
1    0.060714
9    0.001623
Name: conf_covid1, dtype: float64

In [29]:
data['mis_covid2'].value_counts(normalize=True)

2    0.794156
1    0.204545
9    0.001299
Name: mis_covid2, dtype: float64

In [30]:
data['conf_covid2'].value_counts(normalize=True)

5    0.378247
4    0.241558
3    0.235390
2    0.099675
1    0.042857
9    0.002273
Name: conf_covid2, dtype: float64

In [31]:
data['viol3_dems'].value_counts(normalize=True)

6    0.613312
4    0.096753
2    0.084416
5    0.081169
1    0.067532
3    0.056494
9    0.000325
Name: viol3_dems, dtype: float64

In [32]:
data['viol3_reps'].value_counts(normalize=True)

6    0.523701
2    0.110714
4    0.110065
3    0.094805
1    0.081169
5    0.079221
9    0.000325
Name: viol3_reps, dtype: float64

In [33]:
data['viol3_both'].value_counts(normalize=True)

6    0.890909
3    0.024351
4    0.023377
1    0.020779
5    0.020455
2    0.020130
Name: viol3_both, dtype: float64

In [34]:
data['facebook1'].value_counts(normalize=True)

1     0.326299
66    0.202597
2     0.179221
3     0.130195
4     0.074351
5     0.034740
7     0.027597
6     0.025000
Name: facebook1, dtype: float64

In [35]:
#need to drop 66
data['twitter1'].value_counts(normalize=True)

66    0.578571
1     0.110390
2     0.078247
3     0.063961
4     0.057468
7     0.043506
6     0.036688
5     0.031169
Name: twitter1, dtype: float64